## Meteo forcing example

This is an example of reading locally stored ECMWF grib files. 

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.meteo as pm
import datetime
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import pandas as pd
from glob import glob
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
#optional
#plt.style.use(['dark_background'])

In [ ]:
from hvplot import xarray

In [ ]:
pyPoseidon.__version__

### setup

In [ ]:
#define in a dictionary the properties of the model..
dic={'lon_min':-25., # lat/lon window
     'lon_max':-9.,
     'lat_min':56.,
     'lat_max':74.,
     'start_date':'2018-10-1',
     'end_date':'2018-10-2'
    }

In [ ]:
DATA_PATH = os.path.dirname(pyPoseidon.__file__)   # get path to the test data installed with the package
DATA_PATH

In [ ]:
# This just a structure to create the list of paths for the actual meteo files - change appropriately 
PATH=DATA_PATH + '/tests/data/'#Path to meteo files
mpaths = glob(PATH+'uvp*.grib')
mpaths.sort()
mpaths

In [ ]:
dic.update({'meteo_source':mpaths, 'meteo_engine': 'cfgrib', 'meteo_merge':'last', 'meteo_combine_by' : 'nested', 'meteo_xr_kwargs':{'concat_dim':'step'} }) # The files are forecasting [0-72] so we combine them in terms of time

### retrieve UVP

In [ ]:
#get uvp
meteo = pm.meteo(**dic)

In [ ]:
meteo.Dataset

## Visualise

In [ ]:
meteo.Dataset.msl[0,:,:].plot()

### holoviews

In [ ]:
meteo.Dataset.msl.hvplot.contourf(x='longitude',y='latitude',cmap='viridis')

### geoviews

In [ ]:
g50 = gf.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
meteo.Dataset.msl

In [ ]:
meteo.Dataset.msl.hvplot(x='longitude',y='latitude',geo=True, rasterize=True, cmap='viridis', width=800, height=600) * g50 * gf.borders()

### Animation

In [ ]:
from pyPoseidon.utils.pplot import contour # based on matplotlib and ffmpeg

In [ ]:
contour(meteo.Dataset.longitude.values,meteo.Dataset.latitude.values,meteo.Dataset.msl.values,meteo.Dataset.time.values)

## Output to file

In [ ]:
#generic netCDF output
meteo.Dataset.to_netcdf('./test/test.nc') # to netcdf

In [ ]:
#save to DELFT3D format
meteo.to_output(solver='d3d',rpath='./test/') # to u,v,p for d3d

In [ ]:
#save to SCHISM format
meteo.to_output(solver='schism',rpath='./test/')